# 2 Single 2 Perceptron

In [1]:
# Package imports

from recorder_util import ModelResults
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from nltk.tokenize import word_tokenize

In [2]:
# Initializing recorder results

model_name = "2_Single_2Perceptron_Word2Vec"
author = "Noah Sher"
seed = 123

recorder = ModelResults(model_name, author, seed)

In [3]:
# Loading datasets

train_data = pd.read_csv("trainData.csv")
test_data = pd.read_csv("testData.csv")

print("Unprocessed data:")
train_data.head()

Unprocessed data:


,label,source,text
0,0,1,Help wanted!\n\nThe Seagoing Cowboys program i...
1,0,1,The system of the Electoral College is a widel...
2,1,1,The renowned British statesman Winston Churchi...
3,0,1,"My grandfather would always say ""creativity is..."
4,1,1,In my pursuit to become an assistant manager a...


In [4]:
# Preprocessing text by:
#   1. lowercasing all letters and
#   2. splitting all strings into a list of words.

def preprocess_data(text):
    lower_text = text.lower()
    preprocessed = lower_text.split()
    return preprocessed

train_data["preprocessed_text"] = train_data["text"].apply(preprocess_data)
test_data["preprocessed_text"] = test_data["text"].apply(preprocess_data)

print("Preprocessed data:")
train_data.head()

Preprocessed data:


,label,source,text,preprocessed_text
0,0,1,Help wanted!\n\nThe Seagoing Cowboys program i...,"[help, wanted!, the, seagoing, cowboys, progra..."
1,0,1,The system of the Electoral College is a widel...,"[the, system, of, the, electoral, college, is,..."
2,1,1,The renowned British statesman Winston Churchi...,"[the, renowned, british, statesman, winston, c..."
3,0,1,"My grandfather would always say ""creativity is...","[my, grandfather, would, always, say, ""creativ..."
4,1,1,In my pursuit to become an assistant manager a...,"[in, my, pursuit, to, become, an, assistant, m..."


In [ ]:
# Training Word2Vec on the train_data

word2Vec = Word2Vec(
    sentences = train_data["text"],
    vector_size = 100,
    window = 5,
    min_count = 1,
    workers = 4
)

def embeddings(text, model):
    vectors = []
    for word in text:
        if word in model.wv:
            vectors.append(model.wv[word])
    if vectorsrs:
        return np.mean(vectors, axis = 0)
    else:
        return np.zeros(model.vector_size)

In [ ]:
# Converting training texts to embeddings

x_train = []
for text in train_data["preprocessed_text"]:
    embedding = embeddings(text, word2Vec)
    x_train.append(embedding)
x_train = np.array(x_train)
y_train = train_data["label"].values

In [ ]:
# Converting testing texts to embeddings

x_test = []
for text in test_data["preprocessed_text"]:
    embedding = emeddings(text, word2Vec)
    x_test.append(embedding)
x_test = np.array(x_test)
y_test = test_data["label"].values

In [ ]:
# initializing weights and bias

np.random.seed(seed)
n_examples, n_features = x_train_scaled.shape
w = np.zeros(n_features)
b = 0

In [ ]:
# Training the perceptron

n_epochs = 10
indices = np.arange(n_examples)
recorder.record_training_start()
for epoch in range(10):
    n_errors = 0
    # randomize the order in which training examples are seen in this epoch
    np.random.shuffle(indices)
    # traverse the training data
    for i in tqdm(indices, desc = f'epoch {epoch + 1}'):
        x = X_train[i]
        y_true = y_train[i]
        # the perceptron decision based on the current model
        score = x @ w + b
        y_pred = 1 if score > 0 else 0
        # update the model is the prediction was incorrect
        if y_true == y_pred:
            continue
        elif y_true == 1 and y_pred == 0:
            w = w + x
            b = b + 1
            n_errors += 1
        elif y_true == 0 and y_pred == 1:
            w = w - x
            b = b - 1
            n_errors += 1
    print(f"Epoch {epoch + 1}: Errors = {n_errors}")
    if n_errors == 0:
        break
recorder.record_training_stop()

In [ ]:
# Testing the perceptron

recorder.record_testing_start()
y_pred = (np.dot(x_test_scaled, w) + b) > 0
recorder.record_testing_stop()

In [ ]:
def binary_classification_report(y_true, y_pred):
    # count true positives, false positives, true negatives, and false negatives
    tp = fp = tn = fn = 0
    for gold, pred in zip(y_true, y_pred):
        if pred == True:
            if gold == True:
                tp += 1
            else:
                fp += 1
        else:
            if gold == False:
                tn += 1
            else:
                fn += 1
    # calculate precision and recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    # calculate f1 score
    fscore = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    # calculate accuracy
    accuracy = (tp + tn) / len(y_true)
    # number of positive labels in y_true
    support = sum(y_true)
    return {
        "precision": precision,
        "recall": recall,
        "f1-score": fscore,
        "support": support,
        "accuracy": accuracy,
    }

In [ ]:
# Recording results

binary_classification_report(y_test, y_pred)
recorder.record_performance(
    y_true=y_test,
    y_pred=y_pred,
    target_names=["Not Bot", "Bot"],
)
recorder.write("model_results.csv")